In [1]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

2023-11-11 18:13:34.215870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
ratings_df = pd.read_csv('./dataset_out/out_user_movie_rating.csv')

ratings_df.head()

,userId,movieId,rating,date,original_title,genres,overview
0,89,3190,5,2023-11-06 16:10:58,Childish Quarrel,"[""Documentary""]",Two babies are shown seated next to each other...
1,73,3190,4,2023-11-06 16:20:33,Childish Quarrel,"[""Documentary""]",Two babies are shown seated next to each other...
2,56,3190,8,2023-11-06 16:23:49,Childish Quarrel,"[""Documentary""]",Two babies are shown seated next to each other...
3,32,3190,3,2023-11-06 16:33:25,Childish Quarrel,"[""Documentary""]",Two babies are shown seated next to each other...
4,142,3190,3,2023-11-06 16:34:15,Childish Quarrel,"[""Documentary""]",Two babies are shown seated next to each other...


In [3]:
movies_df = pd.read_csv('./dataset_out/out_simple_movie_infos.csv')

movies_df.head()

,movieId,original_title
0,3190,Childish Quarrel
1,2790,Automated Hat-Maker and Sausage-Grinder
2,3548,A Story Well Spun
3,2823,National parade to celebrate the annexation of...
4,2980,A Pair of Silk Stockings


In [4]:
ratings_df['userId'] = ratings_df['userId'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df[['userId', 'original_title', 'rating']]))
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df[['original_title']]))

ratings = ratings.map(lambda x: {
    "original_title": x["original_title"],
    "userId": x["userId"],
    "rating": float(x["rating"])
})

movies = movies.map(lambda x: x["original_title"])

In [5]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(37_000)
test = ratings.skip(35_000).take(8_282)

Total Data: 45282


In [8]:
# Extract unique movie titles and user IDs
unique_movie_titles = np.unique([x["original_title"].numpy().decode("utf-8") for x in train.map(lambda x: x)])
unique_user_ids = np.unique([x["userId"].numpy().decode("utf-8") for x in train.map(lambda x: x)])


2023-11-11 18:14:24.069813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [45282]
	 [[{{node Placeholder/_2}}]]
2023-11-11 18:14:28.210504: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [45282]
	 [[{{node Placeholder/_2}}]]


In [9]:
from typing import Dict, Text
class MovieModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight
    

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["userId"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["original_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )


  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [10]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3


2023-11-11 18:14:40.527326: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [45282]
	 [[{{node Placeholder/_0}}]]


 2/37 [>.............................] - ETA: 3s - root_mean_squared_error: 5.4053 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0010 - factorized_top_k/top_10_categorical_accuracy: 0.0025 - factorized_top_k/top_50_categorical_accuracy: 0.0210 - factorized_top_k/top_100_categorical_accuracy: 0.0375 - loss: 6937.1350 - regularization_loss: 0.0000e+00 - total_loss: 6937.1350  

2023-11-11 18:14:42.515318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_12}}]]
2023-11-11 18:14:42.620975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_12}}]]
2023-11-11 18:14:42.714374: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and sha

 5/37 [===>..........................] - ETA: 2s - root_mean_squared_error: 5.8289 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 6.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0014 - factorized_top_k/top_50_categorical_accuracy: 0.0154 - factorized_top_k/top_100_categorical_accuracy: 0.0308 - loss: 6941.9815 - regularization_loss: 0.0000e+00 - total_loss: 6941.9815

...

2023-11-11 18:14:52.272229: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype resource
	 [[{{node Placeholder/_8}}]]
2023-11-11 18:14:52.364383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [3150]
	 [[{{node Placeholder/_0}}]]
2023-11-11 18:14:52.456426: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [1000,64]
	 [[{{node

35/37 [===========================>..] - ETA: 0s - root_mean_squared_error: 2.9353 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0020 - factorized_top_k/top_10_categorical_accuracy: 0.0059 - factorized_top_k/top_50_categorical_accuracy: 0.0751 - factorized_top_k/top_100_categorical_accuracy: 0.2001 - loss: 6308.7575 - regularization_loss: 0.0000e+00 - total_loss: 6308.7575

2023-11-11 18:14:52.548820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]
2023-11-11 18:14:52.640566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]
2023-11-11 18:14:52.730157: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [3150]
	 [[{{node Placeholder/_0}}]

37/37 [==============================] - 3s 92ms/step - root_mean_squared_error: 2.9441 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0019 - factorized_top_k/top_10_categorical_accuracy: 0.0057 - factorized_top_k/top_50_categorical_accuracy: 0.0738 - factorized_top_k/top_100_categorical_accuracy: 0.1978 - loss: 6306.9303 - regularization_loss: 0.0000e+00 - total_loss: 6306.9303


2023-11-11 18:14:52.823061: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64
	 [[{{node Placeholder/_7}}]]


In [11]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

2023-11-11 18:14:52.917233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [45282]
	 [[{{node Placeholder/_1}}]]


2/9 [=====>........................] - ETA: 0s - root_mean_squared_error: 2.9981 - factorized_top_k/top_1_categorical_accuracy: 0.0050 - factorized_top_k/top_5_categorical_accuracy: 0.0345 - factorized_top_k/top_10_categorical_accuracy: 0.0630 - factorized_top_k/top_50_categorical_accuracy: 0.2665 - factorized_top_k/top_100_categorical_accuracy: 0.4690 - loss: 5760.0437 - regularization_loss: 0.0000e+00 - total_loss: 5760.0437

2023-11-11 18:14:53.466106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [3150]
	 [[{{node Placeholder/_0}}]]
2023-11-11 18:14:53.555478: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_12}}]]
2023-11-11 18:14:53.643688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [

5/9 [===============>..............] - ETA: 0s - root_mean_squared_error: 2.9627 - factorized_top_k/top_1_categorical_accuracy: 0.0020 - factorized_top_k/top_5_categorical_accuracy: 0.0138 - factorized_top_k/top_10_categorical_accuracy: 0.0252 - factorized_top_k/top_50_categorical_accuracy: 0.1066 - factorized_top_k/top_100_categorical_accuracy: 0.1876 - loss: 6454.2234 - regularization_loss: 0.0000e+00 - total_loss: 6454.2234

2023-11-11 18:14:53.735911: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [3150]
	 [[{{node Placeholder/_0}}]]
2023-11-11 18:14:53.824477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_12}}]]
2023-11-11 18:14:53.913792: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [

8/9 [=========================>....] - ETA: 0s - root_mean_squared_error: 2.9645 - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0086 - factorized_top_k/top_10_categorical_accuracy: 0.0158 - factorized_top_k/top_50_categorical_accuracy: 0.0666 - factorized_top_k/top_100_categorical_accuracy: 0.1173 - loss: 6627.6009 - regularization_loss: 0.0000e+00 - total_loss: 6627.6009

2023-11-11 18:14:54.005468: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [3150]
	 [[{{node Placeholder/_0}}]]
2023-11-11 18:14:54.097826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_12}}]]
2023-11-11 18:14:54.183821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype resource
	 [[{{no

9/9 [==============================] - 1s 86ms/step - root_mean_squared_error: 2.9609 - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0083 - factorized_top_k/top_10_categorical_accuracy: 0.0152 - factorized_top_k/top_50_categorical_accuracy: 0.0644 - factorized_top_k/top_100_categorical_accuracy: 0.1133 - loss: 5621.9150 - regularization_loss: 0.0000e+00 - total_loss: 5621.9150

Retrieval top-100 accuracy: 0.113
Ranking RMSE: 2.961


In [14]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

In [17]:
predict_movie(1, 20)


Top 20 recommendations for user 1:

1. Querelle
2. Imagine Me & You
3. Dinosaur
4. Hugo
5. All About 'Abigail's Party'
6. Young and Dangerous 2
7. Marlene Dietrich: Her Own Song
8. Daydream Obsession 3: Legacy
9. Apocalypto
10. Doctor Who: Carnival of Monsters


2023-11-11 18:15:38.254596: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [3150]
	 [[{{node Placeholder/_0}}]]
2023-11-11 18:15:38.254855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]


In [34]:
model.retrieval_task = tfrs.tasks.Retrieval()  # Removes the metrics.
model.compile()
model.save("./models/tfrs_model")

INFO:tensorflow:Assets written to: ./models/tfrs_model/assets


INFO:tensorflow:Assets written to: ./models/tfrs_model/assets


In [35]:
tfrs_model = tf.keras.models.load_model("./models/tfrs_model")

In [36]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(tfrs_model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(tfrs_model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))
predict_movie(1, 20)


Top 20 recommendations for user 1:

1. Querelle
2. Imagine Me & You
3. Dinosaur
4. Hugo
5. All About 'Abigail's Party'
6. Young and Dangerous 2
7. Marlene Dietrich: Her Own Song
8. Daydream Obsession 3: Legacy
9. Apocalypto
10. Doctor Who: Carnival of Monsters


2023-11-11 19:19:30.459774: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string and shape [3150]
	 [[{{node Placeholder/_6}}]]
2023-11-11 19:19:30.460956: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]
